# GARLIC demo

General-purpose Adaptive Richardson-Lucy Image Characterisation

# 1. Init

## System setup

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator

import ipywidgets as widgets
from IPython.display import display

import numpy as np
from time import time
from scipy import ndimage, special


In [ ]:
import importlib  # will not be needed in production
import scripts

## Read data

In [ ]:
importlib.reload(scripts.read_data)
object_name, data, beam_FWHM_pix, true_signal = scripts.read_data.run(23, (0, 0, 1))

## GARLIC parameters

In [ ]:
'''
if data.ndim == 1:
    src_radius = 2
elif data.ndim == 2:
    src_radius = 2
elif data.ndim == 3:
    src_radius = [4, 2, 2]
    src_radius = 2
else:
    print('ERROR:', data.ndim, 'dimensions?')
    raise -1
'''

src_radius = beam_FWHM_pix / np.sqrt(8*np.log(2))
bg_radius = np.sqrt(src_radius * np.array(data.shape))
bg_radius = np.fmax(bg_radius, np.power(np.count_nonzero(~np.isfinite(data)), 1/data.ndim))
print(src_radius, bg_radius)

# 2. Signal, background and noise

In [ ]:
importlib.reload(scripts.utils)
signal = scripts.utils.weighted_gaussian_filter(data, src_radius)
p16, p50, p84 = np.nanpercentile(signal, [16, 50, 84])
print(p16, p50, p84)
bg_mask = np.exp(np.clip(-(signal - p50) / (p50 - p16), -300, 300))
bg_mask /= scripts.utils.weighted_gaussian_filter(bg_mask, bg_radius)
#bg_mask = np.clip(2*(bg_mask - .5), 0, 1)
bg_mask = np.clip(bg_mask, 0, 1)**2
background = scripts.utils.weighted_gaussian_filter(signal, bg_radius, weight=bg_mask)

In [ ]:
noise = np.sqrt(scripts.utils.weighted_gaussian_filter((data - signal)**2, bg_radius, weight=bg_mask))

In [ ]:
signal -= background

In [ ]:
SN = signal / noise

## Test (1D)

In [ ]:
if data.ndim == 1:
    fig, axes = scripts.utils.new_figure('signal_background_noise', nrows=2)


    ax = axes[0, 0]
    ax.plot(data, 'k-', alpha=.5, label='data')
    if true_signal is not None:
        ax.plot(true_signal, 'k-', label='true signal')


    ax.plot(background + signal, 'b-', alpha=.5, label=f'signal (R={np.array_str(np.array(src_radius), precision=2)} pix)')
    ax.fill_between(np.arange(data.size), background + signal - noise, background + signal + noise,
                    color='b', alpha=.1, label=f'noise: {np.mean(noise):.3g}$\\pm${np.std(noise):.3g}')  # [{np.min(noise):.3g} - {np.max(noise):.3g}]')

    ax.plot(background, 'k--', alpha=.5, label=f'background (R={np.array2string(np.array(bg_radius), precision=2)} pix)')

    ax.legend()
    #ax.set_ylim(0., 500)
    
    
    ax = axes[1, 0]
    ax.set_ylabel('S/N')
    ax.plot(SN, 'k-')

    ax.plot(bg_mask, 'r-')
    #ax.set_ylim(-.1, 1.1)
    #ax.set_ylim(0., 500)
    #ax.set_yscale('log')
    plt.show()

## Test (2D)

In [ ]:
if data.ndim == 2:
    fig, axes = scripts.utils.new_figure('signal_background_noise', nrows=2, ncols=3,
                                         figsize=(12, 6),
                                         sharex=True, sharey=True, gridspec_kw={})


    ax = axes[0, 0]
    norm = scripts.utils.colour_map(ax, 'data', data)[2]
    #ax.contour(SN - bg_mask, levels=[0], colors=['w'])
    ax.contour(SN, levels=[1], colors=['k'], linestyles=['-'])

    ax = axes[0, 1]
    scripts.utils.colour_map(ax, f'signal (R={np.array_str(np.array(src_radius), precision=2)} pix)', signal)

    ax = axes[0, 2]
    scripts.utils.colour_map(ax, 'S/N', signal/noise, cmap='seismic_r', norm=colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200))
    #ax.contour(SN - bg_mask, levels=[0], colors=['k'])


    ax = axes[1, 0]
    scripts.utils.colour_map(ax, f'background (R={np.array2string(np.array(bg_radius), precision=2)} pix)', background, norm=norm)

    ax = axes[1, 1]
    scripts.utils.colour_map(ax, 'noise', noise, cmap='inferno')

    ax = axes[1, 2]
    scripts.utils.colour_map(ax, 'bg_mask', bg_mask, cmap='seismic')


    plt.show()

## Test (3D)

In [ ]:
if data.ndim == 3:
    fig, axes = scripts.utils.new_figure('bg_test', nrows=2, ncols=3,
                                         figsize=(12, 6), sharex=True, sharey=True, gridspec_kw={})

    ax = axes[0, 0]
    norm = scripts.utils.colour_map(ax, 'data', data, projection_axis=0)[2]
    #ax.contour(np.max(mask.astype(int)*signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
    #ax.contour(np.max(mask*SN, axis=0), levels=[peak_threshold], colors=['r'], linestyles=['-'])
    #ax.contour(np.max(SN, axis=0), levels=[1], colors=['k'], linestyles=[':'])
    if true_signal is not None:
        ax.contour(np.max(true_signal/noise, axis=0), levels=[1], colors=['k'], linestyles=['-'])

    ax = axes[0, 1]
    scripts.utils.colour_map(ax, 'signal', signal, projection_axis=0, norm=norm)
    if true_signal is not None:
        ax.contour(np.max(true_signal/noise, axis=0), levels=[1], colors=['k'], linestyles=['-'])

    ax = axes[0, 2]
    scripts.utils.colour_map(ax, 'S/N', SN, projection_axis=0,
                             cmap='seismic_r', norm=colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200))
    #ax.contour(np.max(mask.astype(int)*signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
    #ax.contour(np.max(mask*SN, axis=0), levels=[peak_threshold], colors=['w'], linestyles=['-'])
    #if true_signal is not None:
    #    ax.contour(np.max(true_signal/noise, axis=0), levels=[.5, 1], colors=['k', 'y'])

    ax = axes[1, 0]
    scripts.utils.colour_map(ax, 'background', background, projection_axis=0, norm=norm)

    ax = axes[1, 1]
    scripts.utils.colour_map(ax, 'noise', noise, cmap='inferno', projection_axis=0)

    ax = axes[1, 2]
    if true_signal is not None:
        scripts.utils.colour_map(ax, 'true S/N', true_signal/noise, projection_axis=0,
                                 cmap='seismic_r', norm=colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200))
    ax.contour(np.max(SN, axis=0), levels=[1], colors=['k'])
    #ax.contour(np.max(SN - bg_mask, axis=0), levels=[0], colors=['k'])
    #ax.contour(np.max(SN, axis=0), levels=[.5, 1], colors=['k', 'y'])

    plt.show()

In [ ]:
if data.ndim == 3:
    fig, axes = scripts.utils.new_figure('single_spaxel', figsize=(12, 12), nrows=7, sharex='col', sharey=False)

    index_y = np.random.randint(0, data.shape[1]); index_x = np.random.randint(0, data.shape[2])
    #index_y = 132; index_x = 137
    #index_y = 128; index_x = 142
    index_y = 30; index_x = 37
    #index_y = 30; index_x = 43
    #index_y = 33; index_x = 139
    #index_y = 21; index_x = 143

    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    vmin, vmax = np.nanpercentile(data, [16, 84])
    norm = colors.Normalize(vmin, vmax)
    SN_norm = colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200)

    ax = axes[0, 0]
    if true_signal is not None:
        scripts.utils.colour_map(ax, 'true S/N', (true_signal/noise)[:, index_y, :].T, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    ax.axhline(index_x, c='w', ls='--')
    ax.set_ylabel('x')

    ax = axes[1, 0]
    ax.get_shared_y_axes().join(ax, axes[0, 0])
    SN_map = (signal/noise)[:, index_y, :].T
    scripts.utils.colour_map(ax, 'S/N', SN_map, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    ax.axhline(index_x, c='w', ls='--')
    ax.contour(SN_map, levels=[1], colors=['k'], linestyles=['-'])
    #axes[0, 0].contour(SN_map - bg_mask[:, index_y, :].T, levels=[0], colors=['w'])
    axes[0, 0].contour(SN_map, levels=[1], colors=['k'], linestyles=['-'])
    ax.set_ylabel('x')

    ax = axes[2, 0]
    ax.get_shared_y_axes().join(ax, axes[0, 0])
    scripts.utils.colour_map(ax, 'observed', data[:, index_y, :].T, projection_axis=None, norm=norm)
    ax.axhline(index_x, c='w', ls='--')
    ax.set_ylabel('x')


    ax = axes[3, 0]
    ax.plot(data[:, index_y, index_x], 'k-', alpha=.2, label='measured')
    if true_signal is not None:
        ax.plot(true_signal[:, index_y, index_x], 'k-', label='true')
    ax.plot(signal[:, index_y, index_x], 'b-', alpha=.5, label='signal')
    ax.plot(background[:, index_y, index_x], 'r--', alpha=.5, label='background')
    ax.fill_between(np.arange(data.shape[0]),
                    (background - noise)[:, index_y, index_x],
                    (background + noise)[:, index_y, index_x],
                    color='r', alpha=.1,
                    label=f'noise: {np.nanmean(noise):.3g}$\\pm${np.std(noise):.3g}') # [{np.min(noise):.3g} - {np.max(noise):.3g}]')
    ax.set_ylabel('intensity')
    ax.legend()
    cb = fig.colorbar(None, ax=ax, orientation='vertical', shrink=.9)



    ax = axes[4, 0]
    ax.get_shared_y_axes().join(ax, axes[-1, 0])
    scripts.utils.colour_map(ax, 'observed', data[:, :, index_x].T, projection_axis=None, norm=norm)
    ax.axhline(index_y, c='w', ls='--')
    ax.set_ylabel('y')

    ax = axes[5, 0]
    ax.get_shared_y_axes().join(ax, axes[-1, 0])
    SN_map = (signal/noise)[:, :, index_x].T
    scripts.utils.colour_map(ax, 'S/N', SN_map, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    ax.axhline(index_y, c='w', ls='--')
    ax.contour(SN_map, levels=[1], colors=['k'])
    #ax.contour(SN_map, levels=[0.5, 1], colors=['k', 'y'])
    ax.set_ylabel('y')

    ax = axes[6, 0]
    if true_signal is not None:
        scripts.utils.colour_map(ax, 'true S/N', (true_signal/noise)[:, :, index_x].T, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    #scripts.utils.colour_map(ax, 'bg_mask', bg_mask[:, :, index_x].T, projection_axis=None, cmap='seismic', norm=SN_norm)#, norm=colors.Normalize())
    ax.contour(SN_map, levels=[1], colors=['k'])
    #ax.contour(SN_map, levels=[0.5, 1], colors=['k', 'y'])
    #ax.contour(SN_map - bg_mask[:, :, index_x].T, levels=[0], colors=['w'])
    ax.axhline(index_y, c='w', ls='--')
    ax.set_ylabel('y')

    ax.set_xlabel('channel')

    plt.show()

# --- STOP ---

In [ ]:
raise -1

# 2. Adaptive Richardson Lucy

Convergence:

In [ ]:
residual_accuracy = .01
max_iter = 10
kernel_truncation = 6

Smoothing scales

In [ ]:
'''
noise_scale = scripts.diffuse_emission.find_scale(data)
source_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, noise_scale, truncate=kernel_truncation))
diffuse_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, source_scale, truncate=kernel_truncation))
print(f'Scales: noise = {noise_scale:.2f}, sources = {source_scale:.2f}, diffuse emission = {diffuse_scale:.2f}')
'''
#src_radius = [4, 2, 2]
src_radius = 2
if data.ndim == 1:
    bg_radius = np.sqrt(src_radius * data.size)
else:
    bg_radius = np.sqrt(src_radius * np.array(data.shape))
#bg_radius = max(10 * src_radius,
#                np.sqrt(src_radius * np.array(data.shape)))
#bg_radius = (src_radius + np.array(data.shape)) / 2
#bg_radius = 1000
print(f'Scales: sources = {np.array_str(np.array(src_radius), precision=2)}, diffuse emission = {np.array_str(np.array(bg_radius), precision=2)}')


## Deconvolution

In [ ]:
importlib.reload(scripts.multiscale_RL)
baseline = np.nanmin(data)
smoothing_radii = np.array([src_radius, bg_radius])
n_radii = smoothing_radii.shape[0]
mRL = scripts.multiscale_RL.run(data, smoothing_radii, residual_accuracy=residual_accuracy, max_iter=100, kernel_truncation=kernel_truncation)
RL = np.sum(mRL, axis=0)
m_model = np.empty_like(mRL)
for i, radius in enumerate(smoothing_radii):
    m_model[i] = ndimage.gaussian_filter(mRL[i], radius, truncate=kernel_truncation)
model = baseline + np.sum(m_model, axis=0)

In [ ]:
if data.ndim == 2:
    importlib.reload(scripts.utils)
    fig, axes = scripts.utils.new_figure('RL_reconstruction', nrows=3, ncols=3,
                                         figsize=(12, 8), sharex=True, sharey=True, gridspec_kw={})


    ax = axes[0, 0]
    norm = scripts.utils.colour_map(ax, 'data', data)[2]

    ax = axes[0, 1]
    scripts.utils.colour_map(ax, 'R-L background', baseline+2*mRL[1], norm=norm)

    ax = axes[0, 2]
    scripts.utils.colour_map(ax, 'R-L sources', baseline+2*mRL[0], norm=norm)

    
    ax = axes[1, 0]
    scripts.utils.colour_map(ax, 'model', model, norm=norm)

    ax = axes[1, 1]
    scripts.utils.colour_map(ax, 'model bg', m_model[1], norm=norm)

    ax = axes[1, 2]
    scripts.utils.colour_map(ax, 'model src', m_model[0], norm=norm)

    
    ax = axes[2, 0]
    scripts.utils.colour_map(ax, 'data - model', (data - model) / (np.abs(data) + np.abs(model)), cmap='seismic_r', norm=colors.Normalize(vmin=-1, vmax=1))

    ax = axes[2, 1]
    scripts.utils.colour_map(ax, 'RL - model', (mRL[1] - m_model[1]) / (np.abs(mRL[1]) + np.abs(m_model[1])), cmap='seismic_r', norm=colors.Normalize(vmin=-1, vmax=1))

    ax = axes[2, 2]
    scripts.utils.colour_map(ax, 'RL - model', (mRL[0] - m_model[0]) / (np.abs(mRL[0]) + np.abs(m_model[0])), cmap='seismic_r', norm=colors.Normalize(vmin=-1, vmax=1))
    
    plt.show()

# 3. Image characterisation

## Signal, background and noise

Background

In [ ]:
background = np.fmin(m_model[-1], mRL[-1])
#background = np.min(np.vstack([m_model[-1], mRL]), axis=0)
#background = np.where(m_model[-1] < mRL[-1], mRL[-1], 0)
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = baseline + n_radii*background

In [ ]:
background += baseline + n_radii*m_model[-1]
background /= 2

Noise

In [ ]:
bg_noise = np.sqrt(ndimage.gaussian_filter((baseline + n_radii*m_model[-1] - background)**2, bg_radius, truncate=kernel_truncation))

In [ ]:
noise = np.sqrt(ndimage.gaussian_filter((data - model)**2, src_radius, truncate=kernel_truncation))

In [ ]:
mean_noise = np.nanmean(noise)
noise = np.where(np.isfinite(noise), noise, mean_noise)
print(f'noise: {mean_noise:.3g} +- {np.std(noise):.3g} [{np.min(noise):.3g} - {np.max(noise):.3g}]')

Signal:

In [ ]:
signal = model - background
SN = signal / np.sqrt(noise**2 + bg_noise**2)
median_SN = np.nanmedian(SN)

Test (1D)

In [ ]:
if data.ndim == 1:
    fig, axes = scripts.utils.new_figure('RL_reconstruction', nrows=2)

    ax = axes[0, 0]
    ax.set_ylabel('intensity')

    ax.plot(data, 'k-', alpha=.5, label='data')
    if true_signal is not None:
        ax.plot(true_signal, 'k-', label='true signal')


    #ax.plot(baseline + 2*mRL[0], 'g:', alpha=.5, label='Richardson-Lucy src')
    #ax.plot(baseline + 2*mRL[1], 'y-', alpha=.5, label='Richardson-Lucy bg')
    #ax.plot(baseline + RL, 'r-', alpha=.5, label='Richardson-Lucy')

    ax.plot(baseline + 2*m_model[0], 'g:', alpha=.5, label='model sources')
    ax.plot(baseline + 2*m_model[1], 'y-', alpha=.5, label='model bg')
    ax.plot(model, 'b-', alpha=.5, label=f'model (R={np.array_str(np.array(src_radius), precision=2)} pix)')
    ax.fill_between(np.arange(data.size), model - noise, model + noise,
                    color='b', alpha=.1, label=f'noise: {mean_noise:.3g}$\\pm${np.std(noise):.3g}')  # [{np.min(noise):.3g} - {np.max(noise):.3g}]')

    ax.plot(background, 'k--', alpha=.5, label=f'background (R={np.array2string(np.array(bg_radius), precision=2)} pix)')
    ax.fill_between(np.arange(data.size), background - bg_noise, background + bg_noise,
                    color='k', alpha=.1, label=f'bg noise: {np.mean(bg_noise):.3g}$\\pm${np.std(bg_noise):.3g}')  # [{np.min(noise):.3g} - {np.max(noise):.3g}]')

    ax.legend()
    
    
    ax = axes[1, 0]
    ax.set_ylabel('S/N')
    ax.plot(SN, 'k-')

    #ax.set_ylim(0., 500)
    #ax.set_yscale('log')
    plt.show()

Test (2D)

In [ ]:
if data.ndim == 2:
    importlib.reload(scripts.utils)
    fig, axes = scripts.utils.new_figure('signal_bsckground_noise', nrows=3, ncols=3,
                                         figsize=(12, 8), sharex=True, sharey=True, gridspec_kw={})

    #fig = plt.figure(fig_name, figsize=(8, 5))
    #axes = fig.subplots(nrows=2, ncols=2, squeeze=False, sharex=True, sharey=True)
    #fig.suptitle(fig_name)
    #fig.set_tight_layout(True)


    ax = axes[0, 0]
    norm = scripts.utils.colour_map(ax, 'data', data)[2]

    ax = axes[0, 1]
    scripts.utils.colour_map(ax, 'R-L sources', baseline+2*mRL[0], norm=norm)

    ax = axes[0, 2]
    scripts.utils.colour_map(ax, 'R-L background', baseline+2*mRL[1], norm=norm)

    ax = axes[1, 0]
    norm1 = scripts.utils.colour_map(ax, 'signal', data-background)[2]

    ax = axes[1, 1]
    scripts.utils.colour_map(ax, 'model', model, norm=norm)

    ax = axes[1, 2]
    scripts.utils.colour_map(ax, 'background', background, norm=norm)

    ax = axes[2, 0]
    #colour_map(ax, 'noise', noise, norm=norm)
    #colour_map(ax, 'noise', noise, cmap='inferno')
    scripts.utils.colour_map(ax, 'S/N', SN, cmap='seismic_r', norm=colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200))
    #ax.contour(SN, levels=[peak_threshold], colors=['k'])
    #ax.contour(SN, levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['-', '--'])

    ax = axes[2, 1]
    scripts.utils.colour_map(ax, 'noise', noise, cmap='inferno')

    ax = axes[2, 2]
    scripts.utils.colour_map(ax, 'bg noise', bg_noise, cmap='inferno')
    
    plt.show()

Test (3D)

In [ ]:
if data.ndim == 3:
    importlib.reload(scripts.utils)
    fig, axes = scripts.utils.new_figure('bg_test', nrows=2, ncols=3,
                                         figsize=(12, 6), sharex=True, sharey=True, gridspec_kw={})

    ax = axes[0, 0]
    norm = scripts.utils.colour_map(ax, 'data', data, projection_axis=0)[2]
    #ax.contour(np.max(mask.astype(int)*signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
    #ax.contour(np.max(mask*SN, axis=0), levels=[peak_threshold], colors=['r'], linestyles=['-'])
    ax.contour(np.max(SN, axis=0), levels=[1], colors=['k'], linestyles=[':'])
    if true_signal is not None:
        #ax.contour(np.max(true_signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'])
        ax.contour(np.max(true_signal/noise, axis=0), levels=[1], colors=['k'], linestyles=['-'])

    ax = axes[0, 1]
    scripts.utils.colour_map(ax, 'model', model, projection_axis=0, norm=norm)

    ax = axes[0, 2]
    scripts.utils.colour_map(ax, 'background', background, projection_axis=0, norm=norm)

    ax = axes[1, 0]
    scripts.utils.colour_map(ax, 'S/N', SN, projection_axis=0,
                             cmap='seismic_r', norm=colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200))
    #ax.contour(np.max(mask.astype(int)*signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
    #ax.contour(np.max(mask*SN, axis=0), levels=[peak_threshold], colors=['w'], linestyles=['-'])
    if true_signal is not None:
        #ax.contour(np.max(true_signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'])
        ax.contour(np.max(true_signal/noise, axis=0), levels=[1], colors=['k'], linestyles=['-'])

    ax = axes[1, 1]
    scripts.utils.colour_map(ax, 'noise', noise, cmap='inferno', projection_axis=0)

    ax = axes[1, 2]
    scripts.utils.colour_map(ax, 'bg noise', bg_noise, cmap='inferno', projection_axis=0)

    plt.show()

In [ ]:
if data.ndim == 3:
    importlib.reload(scripts.utils)
    fig, axes = scripts.utils.new_figure('single_spaxel', figsize=(12, 12), nrows=7, sharex='col', sharey=False)

    index_y = np.random.randint(0, data.shape[1]); index_x = np.random.randint(0, data.shape[2])
    #index_y = 132; index_x = 137
    #index_y = 128; index_x = 142
    #index_y = 30; index_x = 37
    #index_y = 30; index_x = 43
    index_y = 33; index_x = 139
    #index_y = 21; index_x = 143

    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    vmin, vmax = np.nanpercentile(data, [16, 84])
    norm = colors.Normalize(vmin, vmax)
    #SN_norm = colors.Normalize(vmin=-peak_threshold, vmax=peak_threshold)
    SN_norm = colors.SymLogNorm(vmin=-2, linthresh=1, vmax=200)

    ax = axes[0, 0]
    #colour_map(ax, 'true', true_spectrum[:, index_y, :].T, projection_axis=None, norm=norm)
    if true_signal is not None:
        scripts.utils.colour_map(ax, 'true S/N', (true_signal/noise)[:, index_y, :].T, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    ax.axhline(index_x, c='w', ls='--')
    ax.set_ylabel('x')

    ax = axes[1, 0]
    ax.get_shared_y_axes().join(ax, axes[0, 0])
    SN_map = (signal/noise)[:, index_y, :].T
    scripts.utils.colour_map(ax, 'S/N', SN_map, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    ax.axhline(index_x, c='w', ls='--')
    #mask = diffuse_source[:, index_y, :].T
    #mask |= compact_source[:, index_y, :].T
    #mask &= SN_map > accretion_threshold
    #ax.contour(mask*SN, levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])#], colors=['w'], alpha=.5)
    #ax.contour(mask*SN_map, levels=[peak_threshold], colors=['w'], linestyles=['-'])
    #axes[0, 0].contour(mask*SN_map, levels=[peak_threshold], colors=['w'], linestyles=['-'])
    axes[0, 0].contour(SN_map, levels=[1], colors=['k'], linestyles=['-'])
    #mask &= SN_map > peak_threshold
    #ax.contour((true_spectrum/noise)[:, index_y, :].T, levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'])
    ax.set_ylabel('x')

    ax = axes[2, 0]
    ax.get_shared_y_axes().join(ax, axes[0, 0])
    scripts.utils.colour_map(ax, 'observed', data[:, index_y, :].T, projection_axis=None, norm=norm)
    ax.axhline(index_x, c='w', ls='--')
    ax.set_ylabel('x')


    ax = axes[3, 0]
    ax.plot(data[:, index_y, index_x], 'k-', alpha=.2, label='measured')
    if true_signal is not None:
        ax.plot(true_signal[:, index_y, index_x], 'k-', label='true')
    ax.plot(model[:, index_y, index_x], 'b-', alpha=.5, label='model')
    ax.plot(background[:, index_y, index_x], 'r--', alpha=.5, label='background')
    ax.fill_between(np.arange(data.shape[0]),
                    (background - noise)[:, index_y, index_x],
                    (background + noise)[:, index_y, index_x],
                    color='r', alpha=.1,
                    label=f'noise: {mean_noise:.3g}$\\pm${np.std(noise):.3g}') # [{np.min(noise):.3g} - {np.max(noise):.3g}]')
    #ax.plot((background + peak_threshold*noise)[:, index_y, index_x], 'k--', label=f'peak_threshold={peak_threshold:.2f}')
    #ax.plot((background + accretion_threshold*noise)[:, index_y, index_x], 'k:', label=f'accretion={accretion_threshold:.2f}', alpha=.5)
    ax.set_ylabel('intensity')
    ax.legend()
    cb = fig.colorbar(None, ax=ax, orientation='vertical', shrink=.9)



    ax = axes[4, 0]
    ax.get_shared_y_axes().join(ax, axes[-1, 0])
    scripts.utils.colour_map(ax, 'observed', data[:, :, index_x].T, projection_axis=None, norm=norm)
    ax.axhline(index_y, c='w', ls='--')
    ax.set_ylabel('y')

    ax = axes[5, 0]
    ax.get_shared_y_axes().join(ax, axes[-1, 0])
    SN_map = (signal/noise)[:, :, index_x].T
    scripts.utils.colour_map(ax, 'S/N', SN_map, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    ax.axhline(index_y, c='w', ls='--')
    #mask = diffuse_source[:, :, index_x].T
    #mask |= compact_source[:, :, index_x].T
    #mask &= SN_map > accretion_threshold
    #ax.contour(mask*SN_map, levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
    #ax.contour(mask*SN_map, levels=[peak_threshold], colors=['w'], linestyles=['-'])
    #mask &= SN_map > peak_threshold
    #ax.contour((true_spectrum/noise)[:, :, index_x].T, levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'], alpha=.5)
    ax.set_ylabel('y')

    ax = axes[6, 0]
    #colour_map(ax, 'true', true_spectrum[:, :, index_x].T, projection_axis=None, norm=norm)
    if true_signal is not None:
        scripts.utils.colour_map(ax, 'true S/N', (true_signal/noise)[:, :, index_x].T, projection_axis=None, cmap='seismic_r', norm=SN_norm)
    #ax.contour(mask*SN_map, levels=[0, peak_threshold], colors=['w', 'w'], linestyles=['--', '-'])
    ax.contour(SN_map, levels=[1], colors=['k'])
    ax.axhline(index_y, c='w', ls='--')
    ax.set_ylabel('y')

    ax.set_xlabel('channel')

    plt.show()

# 3. Source finding

## Hierarchical Overdensity Tree

In [ ]:
importlib.reload(scripts.sort_data)
#argsorted_data, n_valid = scripts.sort_data.run(RL.ravel())
argsorted_data, n_valid = scripts.sort_data.run(mRL.ravel())

In [ ]:
importlib.reload(scripts.HOT)
sorted_strides = np.hstack([np.sort(mRL.strides)//mRL.itemsize, mRL.size]) # DIRTY HACK when testig particles at the boundary
t0 = time()
HOT_labels, HOT_catalog = scripts.HOT.run(mRL, argsorted_data, sorted_strides)
#n_sources = np.unique(HOT_labels).size
n_sources = np.max(HOT_labels)
print(f'     {time()-t0:.3g} seconds')

In [ ]:
HOT_parent = HOT_catalog[0]
'''
HOT_area = HOT_catalog[1]
HOT_test_stat = HOT_catalog[2]
HOT_bg = HOT_catalog[3]
#max_test_stat = catalog[3]
'''
,

## Individual sources

In [ ]:
def get_source_model(mRL, HOT_labels, lbl):
    RL = np.zeros_like(data)
    indices = np.where(HOT_labels[0] == lbl)
    RL[indices] = mRL[0][indices]
    source_model = ndimage.gaussian_filter(RL, smoothing_radii[0])
    
    RL = np.zeros_like(data)
    indices = np.where(HOT_labels[1] == lbl)
    RL[indices] = mRL[1][indices]
    source_model += ndimage.gaussian_filter(RL, smoothing_radii[1])
    
    return source_model

In [ ]:
residual = (compact_emission + diffuse_emission - diffuse_estimate)
def compute_moment(m):
    RL_moment = mRL.copy()
    for i, radius in enumerate(smoothing_radii):
        RL_moment[i] *= ndimage.gaussian_filter(residual**m / (compact_emission + diffuse_emission), radius)
    moment = np.zeros(n_sources+1)
    np.add.at(moment, HOT_labels, RL_moment)
    return moment

source_area = compute_moment(0)
source_residual = compute_moment(1)
mean_residual = source_residual / source_area
#rms_residual = compute_moment(2) / source_area

In [ ]:
source_SN = mean_residual * np.sqrt(source_area) / noise
sorted_by_SN = np.argsort(source_SN)
n_fluke = np.count_nonzero(np.cumsum(source_residual[sorted_by_SN]) < 0)
SN_threshold = source_SN[sorted_by_SN[n_fluke]]
true_source = (source_SN > SN_threshold) & (source_area > compact_scale)
n_true = np.count_nonzero(true_source)
print(f'{n_true} true sourcces above S/N={SN_threshold:.1f} and area={compact_scale:.1f}')

In [ ]:
'''
final_model = baseline + ndimage.gaussian_filter(mRL[1], smoothing_radii[1])

RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])

RL = np.zeros_like(data)
indices = np.where(~true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])
'''

'''
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[1]])
RL[indices] = mRL[1][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[1])
'''

final_model = baseline
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[1]])
RL[indices] = mRL[1][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[1])


In [ ]:
fig, axes = new_figure('source_flux', nrows=1, figsize=(12, 8))

ax = axes[0, 0]
ax.axhline(SN_threshold, color='k', ls=':', label=f'{SN_threshold} x {noise:.3g} = {SN_threshold*noise:.3g}')
ax.axvline(compact_scale, color='k', ls=':', label=f'compact scale = {compact_scale:.3g}')
ax.axvline(diffuse_scale, color='k', ls=':', label=f'diffuse scale = {diffuse_scale:.3g}')

ax.scatter(source_area[true_source], source_SN[true_source], c='b', s=5, alpha=1)
ax.scatter(source_area[~true_source], source_SN[~true_source], c='r', s=5, alpha=.5)
#for i in range(n_sources+1):
#    ax.text(source_residual[i], mean_residual[i], i, fontsize='x-small', clip_on=True)
ax.legend()
plt.show()

# 3. Plots

Normalisation and color maps:

In [ ]:
latin_cube = np.vstack([(np.argsort(np.random.random(n_sources))+1)/n_sources, (np.argsort(np.random.random(n_sources))+1)/n_sources, (np.argsort(np.random.random(n_sources))+1)/n_sources, np.ones(n_sources)]).T
latin_cube[0, :] = [0., 0., 0., 1.]  # background object must be black :^)
label_cmap = colors.ListedColormap(latin_cube)
label_norm = colors.Normalize(vmin=-.5, vmax=n_sources+.5)
print(f'{n_sources} unique sourcces')

In [ ]:
mRL_cmap = default_cmap
mRL_norm = colors.LogNorm(vmin=np.percentile(mRL[mRL>0], 10), vmax=np.percentile(mRL[mRL>0], 99))

In [ ]:
fig_name = 'GARLIC_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12, 10))
axes = fig.subplots(nrows=4, ncols=2, squeeze=False, sharex='col', gridspec_kw={'width_ratios': [1, .02], 'hspace': 0})
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
im = ax.imshow(mRL,
               interpolation='nearest', origin='lower', cmap=mRL_cmap, norm=mRL_norm,
              )
ax.set_aspect('auto')
ax.set_ylabel('smoothing scale')
ax.set_yticks(np.arange(n_radii), [f'{radius:.3g}' for radius in smoothing_radii])
cb = plt.colorbar(im, cax=axes[0, 1], orientation='vertical', shrink=.9)
cb.ax.tick_params(labelsize='small')
cb.ax.set_ylabel('Richardson-Lucy intensity')


ax = axes[1, 0]
im = ax.imshow(source_SN[HOT_labels],
               interpolation='nearest', origin='lower', norm=colors.SymLogNorm(linthresh=SN_threshold), cmap='rainbow_r',
              )
ax.set_aspect('auto')
ax.set_ylabel('smoothing scale')
ax.set_yticks(np.arange(n_radii), [f'{radius:.3g}' for radius in smoothing_radii])
cb = plt.colorbar(im, cax=axes[1, 1], orientation='vertical', shrink=.9)
cb.ax.tick_params(labelsize='small')
cb.ax.set_ylabel('S/N')


ax = axes[2, 0]
masked_labels = HOT_labels.copy()
indices = np.where(~true_source[masked_labels])
masked_labels[indices] = 0
im = ax.imshow(masked_labels,
               interpolation='nearest', origin='lower', cmap=label_cmap, norm=label_norm,
              )
ax.set_aspect('auto')
ax.set_ylabel('smoothing scale')
ax.set_yticks(np.arange(n_radii), [f'{radius:.3g}' for radius in smoothing_radii])
cb = plt.colorbar(im, cax=axes[2, 1], orientation='vertical', shrink=.9)
cb.ax.tick_params(labelsize='small')
cb.ax.set_ylabel('source ID')


ax = axes[3, 0]
ax.set_ylabel('intensity')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
ax.grid(True, which='both')

if true_signal is not None:
    ax.plot(true_signal, 'g-', alpha=.3)
ax.plot(data, 'k-', alpha=.2)
ax.plot(baseline + diffuse_emission + compact_emission, 'b-', alpha=.5)

ax.plot(final_model, 'k-', alpha=1)
ax.plot(2*baseline + compact_emission + diffuse_emission - final_model, 'r-', alpha=1)
ax.plot(baseline + diffuse_estimate, 'g--')
ax.fill_between(np.arange(data.size), baseline + diffuse_estimate - noise, baseline + diffuse_estimate + noise, color='g', alpha=.1)



ax.set_xlabel('pixel ID')
#ax.set_xlim(5900, 6300)
#ax.set_ylim(-10, 300)
#ax.set_ylim(np.min(baseline)*.9, np.max(baseline + diffuse_estimate))

axes[-1, 1].axis('off')
plt.show()

In [ ]:
np.nansum(data), np.nansum(final_model)/np.nansum(data), np.nansum(source_residual), np.nansum(residual), np.nansum(source_residual[~true_source])

In [ ]:
class Explore_lbl_1D(object):
    
    def __init__(self, fig_name, data, estimate, label, parent):
        """Interactive display"""
        
        plt.close(fig_name)
        self.fig = plt.figure(fig_name, figsize=(12, 7))
        self.axes = self.fig.subplots(nrows=4, ncols=2, squeeze=False, sharex='col', gridspec_kw={'width_ratios': [1, .02], 'hspace': 0})
        self.fig.suptitle(fig_name)
        self.fig.set_tight_layout(True)

        self.original = data
        self.data_offset = np.nanmin(data)
        #self.RL = RL
        #self.SSF = SSF
        self.total_estimate = estimate
        self.label = label
        self.parent = parent

        self.ax_parent = self.axes[0, 0]
        self.ax_parent_cb = self.axes[0, 1]
        self.ax_lbl = self.axes[1, 0]
        self.ax_lbl_cb = self.axes[1, 1]

        self.ax_im = self.axes[2, 0]
        self.ax_cb = self.axes[2, 1]

        self.ax0 = self.axes[3, 0]
        self.ax0.plot(data, 'k-', alpha=.2)
        self.ax0.set_xlim(0, data.size)
        self.axes[3, 1].axis('off')

        self.widget = widgets.interactive(self.plot_lbl, lbl=widgets.BoundedIntText(value=0, min=0, max=n_sources, continuous_update=False))
        display(self.widget)


    def plot_lbl(self, lbl):
        #lbl = decreasing_flux[lbl_sorted]
        xlim = self.ax0.get_xlim()
        ylim = self.ax0.get_ylim()

        ax = self.ax_parent
        im = ax.imshow(self.parent[self.label],
                       interpolation='nearest', origin='lower',
                       cmap=label_cmap, norm=colors.Normalize(vmin=-.5, vmax=n_sources+.5),
                      )
        ax.set_aspect('auto')
        ax.set_yticks(np.arange(n_radii), [f'{radius:.3g}' for radius in smoothing_radii])
        cb = plt.colorbar(im, cax=self.ax_parent_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        cb.ax.set_ylabel('parent ID')
        
        
        ax = self.ax_lbl
        im = ax.imshow(self.label,
                       interpolation='nearest', origin='lower',
                       cmap=label_cmap, norm=colors.Normalize(vmin=-.5, vmax=n_sources+.5),
                      )
        ax.set_aspect('auto')
        ax.set_yticks(np.arange(n_radii), [f'{radius:.3g}' for radius in smoothing_radii])
        cb = plt.colorbar(im, cax=self.ax_lbl_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        cb.ax.set_ylabel('source ID')

        
        RL = np.zeros_like(mRL)
        indices = np.where(HOT_labels == lbl)
        RL[indices] = mRL[indices]

        self.ax_im.clear()
        im = self.ax_im.imshow(RL,
                               interpolation='nearest', origin='lower',
                               #cmap='gist_earth', norm=colors.LogNorm(vmin=np.min(RL[RL > 1e3*epsilon])))
                               cmap=default_cmap, norm=colors.LogNorm(vmin=np.percentile(mRL[mRL>0], 10), vmax=np.percentile(mRL[mRL>0], 99)),
                               #cmap=default_cmap, norm=colors.LogNorm(vmin=np.percentile(RL[RL>0], 1), vmax=np.percentile(RL[RL>0], 99)),
                              )
        self.ax_im.set_aspect('auto')
        cb = plt.colorbar(im, cax=self.ax_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        self.fig.canvas.draw_idle()
        

        ax = self.ax0
        ax.clear()

        ax.plot(self.original, 'k-', alpha=.3)
        ax.plot(baseline + diffuse_emission + compact_emission, 'b-', alpha=.25)        
        ax.plot(baseline + diffuse_estimate, 'g--', alpha=.25)
        ax.fill_between(np.arange(data.size), baseline + diffuse_estimate - noise, baseline + diffuse_estimate + noise, color='g', alpha=.1)

        ax.plot(np.sum(RL, axis=0) + baseline, 'r-', alpha=.25, label=f'source {lbl} ({true_source[lbl]})')
        source_fraction = get_source_model(mRL, HOT_labels, lbl) / (compact_emission + diffuse_emission)
        ax.plot(baseline + diffuse_estimate + residual * source_fraction, 'r--',
                label=f'area={source_area[lbl]:.3g} mean={mean_residual[lbl]:.3g} S/N={source_SN[lbl]:.1f}')
        ax.plot((baseline + diffuse_estimate) * source_fraction, 'g--')
        ax.plot(baseline + (compact_emission + diffuse_emission) * source_fraction, 'k-')

        ax.legend()
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')
        #ax.set_yscale('log')
        ax.set_ylim(.9*self.data_offset, 3e5)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        #self.ax0.set_xlim(5000, 5300)


if len(data.shape) == 1:
    estimate = np.empty_like(mRL)
    for i, radius in enumerate(smoothing_radii):
        estimate[i] = ndimage.gaussian_filter(mRL[i], radius)
    estimate = np.sum(estimate, axis=0)

    #for i, radius in enumerate(smoothing_radii):
    #     mRL[i] *= ndimage.gaussian_filter((boosted_data+epsilon) / (estimate+epsilon), radius)
    #RL = np.nansum(mRL, axis=0)
    #SSF_amplitude = np.nanmedian(mRL/RL[np.newaxis, :], axis=1)
    #print(rms_residual)
    x = Explore_lbl_1D(object_name, data, estimate, HOT_labels, HOT_parent)

# --- OLD STUFF ---

In [ ]:
'''
fig_name = 'bg_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12,4))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False)
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
ax.grid(True, which='both')

ax.plot(true_signal, 'g-', alpha=.3)
ax.plot(data, 'k-', alpha=.2)
#ax.axhline(data_offset, c='r', ls=':', alpha=.5)
ax.plot(baseline, 'y-', alpha=.5)
ax.plot(baseline + residual_above_bg, 'y:', alpha=.5)
ax.fill_between(np.arange(data.size), baseline, baseline+2*residual_above_bg, color='y', alpha=.1)


ax.set_xlabel('pixel')
#ax.set_xlim(5900, 6300)
ax.set_ylim(np.min(baseline-residual_above_bg), np.max(baseline+4*residual_above_bg)*1.1)

plt.show()
'''
,

In [ ]:
'''
estimate = baseline + np.sum(mRL, axis=0)
index_minima = scripts.diffuse_emission.find_minima(estimate)

minima = estimate[index_minima] - baseline[index_minima]
p16, p50 = np.nanpercentile(minima, [16, 50])
mu0 = p50
var0 = (p50 - p16)**2
weight = np.exp(-.5 * (minima - mu0)**2 / var0)
total_weight = np.nansum(weight)
mu1 = np.nansum(weight * minima) / total_weight
var1 = np.nansum(weight * (minima- mu1)**2) / total_weight
var = 1 / (1/var1 - 1/var0)
mu = var * (mu1/var1 - mu0/var0)
diffuse_offset = mu
print(diffuse_offset, np.sqrt(var))
'''
,

In [ ]:
'''
importlib.reload(scripts.multiscale_RL)

n_iter = 0
model = data
mean_below = 0
while n_iter < 10 and mean_residual > mean_below:
    n_iter += 1
    compact_scale, diffuse_scale, baseline, residual_above_bg = scripts.diffuse_emission.run(model, max_iter=14)
    smoothing_radii = np.array([1, compact_scale, diffuse_scale])
    print(f'--- {n_iter} --- {compact_scale}, {diffuse_scale}')
    
    mRL = scripts.multiscale_RL.run((model - baseline).clip(min=0), smoothing_radii, .01)
    compact_emission = ndimage.gaussian_filter(mRL[0], 1) + ndimage.gaussian_filter(mRL[1], compact_scale)
    diffuse_residual = ndimage.gaussian_filter(mRL[2], diffuse_scale)
    diffuse_emission = baseline + diffuse_residual
    model = compact_emission + diffuse_emission
    total = np.nansum(data - baseline)
    mean_residual = np.nanmean(diffuse_residual)
    mean_below = np.sqrt(np.nanmean(((baseline - data)**2)[data < baseline]))
    print(np.nansum(compact_emission)/total, mean_residual, mean_below)
'''
,

## Individual sources

RL_flux = np.zeros(n_sources+1)
RL_compact_flux = np.zeros(n_sources+1)
RL_diffuse_flux = np.zeros(n_sources+1)
RL_variance = np.zeros(n_sources+1)
np.add.at(RL_flux, HOT_labels, mRL)
np.add.at(RL_compact_flux, HOT_labels[0], np.max(mRL, axis=0))
#np.add.at(RL_diffuse_flux, HOT_labels[1], mRL[1])
np.add.at(RL_diffuse_flux, HOT_labels[0], np.min(mRL, axis=0))  # dirty hack?
np.add.at(RL_variance, HOT_labels[0], noise**2)


sorted_by_flux = np.argsort(RL_compact_flux)
RL_background = np.sum(mRL, axis=0)
RL_sources = np.zeros_like(data)
model = ndimage.gaussian_filter(RL_background, diffuse_scale)
#chi2 = np.sqrt(np.nanmean(((data - background - model) / noise)**2))
chi2 = np.nanmean(((data - background - model) / noise)**2)
print(chi2, 1+1/np.sqrt(data.size))
for lbl in sorted_by_flux[::-1]:
    old_chi2 = chi2
    indices = np.where(HOT_labels[0] == lbl)
    RL_this_source = np.fmax(mRL[0][indices], mRL[1][indices])
    RL_sources[indices] += RL_this_source
    RL_background[indices] -= RL_this_source
    #RL_sources[indices] += mRL[0][indices]
    #RL_background[indices] -= mRL[0][indices]
    #indices = np.where(HOT_labels[1] == lbl)
    #RL_sources[indices] += mRL[1][indices]
    #RL_background[indices] -= mRL[1][indices]
    model = ndimage.gaussian_filter(RL_background, smoothing_radii[1])
    model += ndimage.gaussian_filter(RL_sources, smoothing_radii[0])
    chi2 = np.sqrt(np.nanmean(((data - background - model) / noise)**2))
    print(lbl, RL_compact_flux[lbl], chi2)
    #if chi2 > old_chi2:
    #    break

sorted_by_flux = np.argsort(RL_compact_flux)
true_source = np.zeros_like(HOT_parent, dtype=bool)
true_source[sorted_by_flux] = np.cumsum(RL_compact_flux[sorted_by_flux]) > RL_flux_threshold

#RL_flux_threshold = np.sum((mRL[1] - mRL[0])[mRL[0] < mRL[1]])
#sorted_by_flux = np.argsort(RL_compact_flux - RL_diffuse_flux)
RL_flux_threshold = np.sum(compact_emission + diffuse_emission - diffuse_estimate)
sorted_by_flux = np.argsort(RL_flux)

true_source = np.zeros_like(HOT_parent, dtype=bool)
true_source[sorted_by_flux[::-1]] = np.cumsum(RL_flux[sorted_by_flux[::-1]]) <= RL_flux_threshold

np.sum(mRL[1]), true_source

RL_background = np.zeros_like(data)
RL_background[~true_source[HOT_labels[0]]] = mRL[0][~true_source[HOT_labels[0]]]
#RL_background[~true_source[HOT_labels[1]]] += mRL[1][~true_source[HOT_labels[1]]]

RL_sources = np.zeros_like(data)
RL_sources[true_source[HOT_labels[0]]] = mRL[0][true_source[HOT_labels[0]]]
RL_sources[true_source[HOT_labels[1]]] += mRL[1][true_source[HOT_labels[1]]]


np.sum(mRL), np.sum(RL_sources), np.sum(RL_background), np.sum(RL_sources + RL_background)

RL_compact_flux[true_source]

fig, axes = new_figure('source_flux', nrows=3, figsize=(12, 8))

ax = axes[0, 0]
ax.plot(RL_compact_flux[sorted_by_flux], np.sqrt(2*np.cumsum(RL_variance[sorted_by_flux])), 'r--')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_compact_flux[sorted_by_flux]), 'b-')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_diffuse_flux[sorted_by_flux]), 'r-')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum((RL_compact_flux - RL_diffuse_flux)[sorted_by_flux]), 'k-')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum((RL_compact_flux - RL_diffuse_flux)[sorted_by_flux]) - np.sqrt(2*np.cumsum(RL_variance[sorted_by_flux])), 'k:')
ax.axhline(np.sum(mRL[1]), c='b', ls=':')
ax.axhline(RL_flux_threshold, c='k', ls='--')
#ax.axhline(np.sum(mRL[0]-mRL[1]), c='orange')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_diffuse_flux[sorted_by_flux]) + np.sqrt(np.cumsum(RL_variance[sorted_by_flux])), c='r', ls=':')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_compact_flux[sorted_by_flux]) - np.sqrt(np.cumsum(RL_variance[sorted_by_flux])), c='b', ls=':')

ax = axes[1, 0]
ax.plot(RL_compact_flux[~true_source], RL_diffuse_flux[~true_source], 'ro')
ax.plot(RL_compact_flux[true_source], RL_diffuse_flux[true_source], 'co')
for i in range(n_sources+1):
    ax.text(RL_compact_flux[i], RL_diffuse_flux[i], i, fontsize='x-small', clip_on=True)
ax.plot([0, np.max(RL_compact_flux)], [0, np.max(RL_compact_flux)], 'k--')

ax = axes[2, 0]
ax.plot(RL_compact_flux[~true_source], RL_compact_flux[~true_source] / np.sqrt(RL_variance[~true_source]), 'ro')
ax.plot(RL_compact_flux[true_source], RL_compact_flux[true_source] / np.sqrt(RL_variance[true_source]), 'co')
for i in range(n_sources+1):
    ax.text(RL_compact_flux[i], RL_compact_flux[i] / np.sqrt(RL_variance[i]), i, fontsize='x-small', clip_on=True)


#raise -1

def get_mRL_inpaint(mRL, labels, target):
    raise -1


def get_mRL_inpaint_OLD(mRL, labels, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices]
    #print(f'mRL {target} (area = {len(indices[0])}): {np.nanmax(mRL_target)} {np.nanmax(mRL[indices])}')

    radius = np.sqrt(mRL.shape[0])
    inpaint_map = np.where(mRL_target > 0, 1., 0.)  # object mask
    if len(inpaint_map[0]) > 0:
        inpaint_map = ndimage.gaussian_filter(inpaint_map, radius)  # interpolation weight
        inpaint_map = np.where(inpaint_map > 0, inpaint_map, np.nan)  # to prevent division by zero
        inpaint_map = ndimage.gaussian_filter(mRL_target, radius) / inpaint_map
    else:
        inpaint_map = mRL_target

    return mRL_target, np.fmin(inpaint_map, mRL)


def get_individual_mRL_OLD(mRL, labels, parent, target):

    mRL_target, inpaint_map = get_mRL_inpaint(mRL, labels, target)
    #print(f'mRL {target}: {np.nanmax(mRL_target)}')
    
    # compute contribution to descendants:
    progenitor = np.array(parent[labels])
    if parent[target] == target:
        indices = np.where(labels == target)
        progenitor[indices] = 0
    n_found = 1
    while n_found > 0:
        indices = np.where(progenitor == target)
        n_found = len(indices[0])
        #print(f'{n_found} values painted for {target} ({np.unique(parent[progenitor][indices])})')
        #print(f'{np.unique(labels[indices])}')
        #print(f'{parent[np.unique(labels[indices])]}')
        mRL_target[indices] = inpaint_map[indices]
        progenitor[indices] = 0
        progenitor = parent[progenitor]

    # remove contribution from ancestors:
    inpaint_map = get_mRL_inpaint(mRL, labels, parent[target])[1]
    indices = np.where(labels == target)
    #print(f'Removing {parent[target]} from {target}: {np.nanmax(mRL_target[indices])} {np.nanmax(inpaint_map[indices])}')
    mRL_target[indices] -= inpaint_map[indices]

    return mRL_target


def get_individual_mRL_another(mRL, labels, parent, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices] - HOT_bg[target]
    # add descendants background:
    progenitor = np.array(parent[labels])
    if parent[target] == target:
        indices = np.where(labels == target)
        progenitor[indices] = 0
        mRL_target[indices] += HOT_bg[target]
    n_found = 1
    while n_found > 0:
        indices = np.where(progenitor == target)
        n_found = len(indices[0])
        #print(f'{n_found} values painted for {target} ({np.unique(parent[progenitor][indices])})')
        #print(f'{np.unique(labels[indices])}')
        #print(f'{parent[np.unique(labels[indices])]}')
        mRL_target[indices] = HOT_bg[labels][indices]
        progenitor[indices] = 0
        progenitor = parent[progenitor]
        n_found = 0  # only one generation
    '''
    '''

def get_individual_mRL(mRL, labels, parent, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices]

    return mRL_target